## 2 Preprocessing

### 0) Setup

In [15]:
import cv2
import numpy as np
from pathlib import Path

### 1) Regolazione del contrasto

In [18]:
# === CONFIGURATION PARAMETERS ===
# Modificare: quando si modificherà il csv dei dataset, fare che lo scorre e prende in automatico
# il path delle immagini con contrasto sotto la soglia
images = [
    "datasets/Playing-Cards-Object-Detection-Dataset/test/images/416724470_jpg.rf.b9479650b738fa89f1eff26bcd5ea65c.jpg",
    "datasets/Playing-Cards-Object-Detection-Dataset/valid/images/349250549_jpg.rf.74ea2da58134f73aa9a3d9f4531ca9bd.jpg",
    "datasets/Playing-Cards-Object-Detection-Dataset/train/images/873020013_jpg.rf.b4818103a66109a0dd85699556ad4ee1.jpg"
]
OUTPUT_DIR = Path('./out')
THRESHOLD = 0.08      # low contrast threshold
METHOD = 'clahe'       # 'clahe', 'stretch', 'both'
CLIP_LIMIT = 3.0
TILE = 50
# ================================

def image_contrast(L: np.ndarray) -> float:
    p2, p98 = np.percentile(L, (2, 98))
    return (p98 - p2) / 255.0  # relative contrast

def stretch_channel(L: np.ndarray, low: float, high: float) -> np.ndarray:
    L = L.astype(np.float32)
    denom = max(high - low, 1e-6)
    L = (L - low) * (255.0 / denom)
    return np.clip(L, 0, 255).astype(np.uint8)

def apply_contrast(img: np.ndarray, method: str, clip_limit: float, tile: int) -> np.ndarray:
    if img.ndim == 2 or img.shape[2] == 1:
        L = img if img.ndim == 2 else img[..., 0]
        if method in ('stretch', 'both'):
            p2, p98 = np.percentile(L, (2, 98))
            L = stretch_channel(L, p2, p98)
        if method in ('clahe', 'both'):
            clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(tile, tile))
            L = clahe.apply(L)
        return L
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    L, a, b = cv2.split(lab)
    if method in ('stretch', 'both'):
        p2, p98 = np.percentile(L, (2, 98))
        L = stretch_channel(L, p2, p98)
    if method in ('clahe', 'both'):
        clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(tile, tile))
        L = clahe.apply(L)
    lab = cv2.merge([L, a, b])
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

def process_image(in_path: Path, out_root: Path) -> tuple[bool, float]:
    img = cv2.imread(str(in_path), cv2.IMREAD_UNCHANGED)
    if img is None:
        return (False, 0.0)
    out_path = out_root / Path(in_path).name
    out_root.mkdir(parents=True, exist_ok=True)

    if img.ndim == 2 or (img.ndim == 3 and img.shape[2] == 1):
        L = img if img.ndim == 2 else img[..., 0]
    else:
        L = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)[:, :, 0]

    contrast = image_contrast(L)
    print(f"new contrast: {contrast}")
    if contrast >= THRESHOLD:
        return (False, contrast)

    out = apply_contrast(img, METHOD, CLIP_LIMIT, TILE)
    cv2.imwrite(str(out_path), out)
    return (True, contrast)

# === EXECUTION ===
total = len(images)
corrected = 0

for img_path in images:
    saved, c = process_image(Path(img_path), OUTPUT_DIR)
    if saved:
        corrected += 1
    print(f"{img_path} -> contrasto {c:.3f} {'(corretto)' if saved else '(ok)'}")

print(f"\nImmagini totali: {total}")
print(f"Corrette (contrast < {THRESHOLD}): {corrected}")
# =================

new contrast: 0.2
datasets/Playing-Cards-Object-Detection-Dataset/test/images/416724470_jpg.rf.b9479650b738fa89f1eff26bcd5ea65c.jpg -> contrasto 0.200 (ok)
new contrast: 0.1568627450980392
datasets/Playing-Cards-Object-Detection-Dataset/valid/images/349250549_jpg.rf.74ea2da58134f73aa9a3d9f4531ca9bd.jpg -> contrasto 0.157 (ok)
new contrast: 0.1843137254901961
datasets/Playing-Cards-Object-Detection-Dataset/train/images/873020013_jpg.rf.b4818103a66109a0dd85699556ad4ee1.jpg -> contrasto 0.184 (ok)

Immagini totali: 3
Corrette (contrast < 0.08): 0


### 2) Immagini con dimensioni differenti

### 3) Bounding Box
Eliminare le label con bounding box:
- più piccole dell'1% dell'immagine.
- più grandi del 90% dell'immagine.
- fuori dai bordi dell'immagine.
Se l'immagine rimane senza label allora eliminarla tutta.

Cercare di avere una buona distribuzione.

### 4) Correggere classi con correlazione troppo grande

### 5) Correzione Label
Controllare se ci sono immagini senza label e il motivo: l'autore del dataset si è sbagliato, oppure perché non c'è alcuna carta. Nel primo caso mettiamo la label, nel secondo cancelliamo l'immagine. 

Inoltre, bisogna controllare anche se il dataset è bilanciato, quindi avere circa lo stesso numero di immagini per ogni label. Questo task è da fare per ultimo perché dipende dai precedenti.